## 1. 安裝必要套件

In [1]:
%pip install torch
%pip install -U transformers==4.57.1 trl==0.25.1 datasets==4.4.1
%pip install matplotlib pandas tqdm

Note: you may need to restart the kernel to use updated packages.
  Using cached datasets-4.4.1-py3-none-any.whl.metadata (19 kB)
Using cached datasets-4.4.1-py3-none-any.whl (511 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 4.2.0
    Uninstalling datasets-4.2.0:
      Successfully uninstalled datasets-4.2.0
  Attempting uninstall: trl━━━━━━━━━━━━━━━━━━━━━ 0/2 [datasets]
    Found existing installation: trl 0.22.2━ 0/2 [datasets]
    Uninstalling trl-0.22.2:━━━━━━━━━━━━━━━━ 0/2 [datasets]
      Successfully uninstalled trl-0.22.2━━━ 0/2 [datasets]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [trl]]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## 2. Hugging Face 登入

請確保你已經：
1. 在 [FunctionGemma 模型頁面](http://huggingface.co/google/functiongemma-270m-it) 接受使用條款
2. 取得 [Access Token](https://huggingface.co/settings/tokens) (需要 'Write' 權限)

In [1]:
import os
from huggingface_hub import login

# 方法 1: 使用環境變數
hf_token = os.environ.get('HF_TOKEN')

# 方法 2: 如果沒有環境變數，手動輸入
if not hf_token:
    hf_token = input("請輸入你的 Hugging Face Token: ")

login(hf_token)
print("✅ 登入成功！")

✅ 登入成功！


## 3. 載入 FunctionGemma 模型

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

gemma_model = "google/functiongemma-270m-it"

# 載入 tokenizer
tokenizer = AutoTokenizer.from_pretrained(gemma_model)

# 載入模型
base_model = AutoModelForCausalLM.from_pretrained(
    gemma_model,
    device_map="auto",
    attn_implementation="eager",
    torch_dtype="auto"
)

print(f"Device: {base_model.device}")
print(f"DType:  {base_model.dtype}")

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/63.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/536M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/176 [00:00<?, ?B/s]

Device: mps:0
DType:  torch.bfloat16


## 4. 載入 Coffee Robot MCP 資料集

載入我們用 Ollama 生成的 `ollama_mcp_dataset.jsonl` 資料集。

資料集包含咖啡機器人外送任務的 Function Calling 範例：
- **baseDrink**: americano, latte, oat_latte, milk
- **floor**: 1~11 樓
- **temperature**: hot, iced
- **addons**: extra_espresso, paper_cup
- **quantity**: 數量

In [3]:
import json
from random import randint
from datasets import Dataset

# 載入本地 JSONL 資料集
dataset_path = "ollama_mcp_dataset.jsonl"

data_list = []
with open(dataset_path, 'r', encoding='utf-8') as f:
    for line in f:
        data_list.append(json.loads(line.strip()))

# 轉換為 Hugging Face Dataset
dataset = Dataset.from_list([{"text": json.dumps(item, ensure_ascii=False)} for item in data_list])
dataset = dataset.shuffle(seed=42)

print(f"📊 資料集大小: {len(dataset)} 筆")
print(f"\n\033[1m資料集範例:\033[0m")
print(json.dumps(json.loads(dataset[randint(0, len(dataset) - 1)]['text']), indent=2, ensure_ascii=False))

📊 資料集大小: 699 筆

資料集範例:
{
  "metadata": "train",
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "create_coffee_robot_mission",
        "description": "建立咖啡機器人外送任務。負責驗證訂單內容，並產生機器人任務指令 (Mock Nuwa Payload)。",
        "parameters": {
          "type": "object",
          "properties": {
            "baseDrink": {
              "type": "string",
              "enum": [
                "americano",
                "latte",
                "oat_latte",
                "milk"
              ],
              "description": "基礎飲品代號，只能是: americano, latte, oat_latte, milk"
            },
            "floor": {
              "type": "integer",
              "description": "送達樓層，必須介於 1 到 11 之間"
            },
            "addons": {
              "type": "array",
              "items": {
                "type": "string",
                "enum": [
                  "extra_espresso",
                  "paper_cup"
                ]
              },
              "descrip

## 5. 資料集格式轉換

將資料集轉換為 TRL SFTTrainer 所需的 Prompt-Completion 格式：
- `prompt`: 不訓練的部分（使用者輸入 + 工具定義）
- `completion`: 要訓練的部分（模型回應）

In [4]:
def apply_format(sample):
    """將資料集格式轉換為 prompt-completion 格式"""
    template_inputs = json.loads(sample['text'])
    
    # 完整的對話（包含 assistant 回應）
    prompt_and_completion = tokenizer.apply_chat_template(
        template_inputs['messages'],
        tools=template_inputs['tools'],
        tokenize=False,
        add_generation_prompt=False
    )
    
    # 只有 prompt 部分（不含 assistant 回應）
    prompt = tokenizer.apply_chat_template(
        template_inputs['messages'][:-1],
        tools=template_inputs['tools'],
        tokenize=False,
        add_generation_prompt=True
    )
    
    # 計算 completion（完整對話 - prompt）
    completion = prompt_and_completion[len(prompt):]
    
    return {
        "prompt": prompt,
        "completion": completion,
        "split": template_inputs.get("metadata", "train"),
    }

# 處理資料集
print("🔄 轉換資料集格式中...")
processed_dataset = dataset.map(apply_format)
print("✅ 資料集格式轉換完成！")

🔄 轉換資料集格式中...


Map:   0%|          | 0/699 [00:00<?, ? examples/s]

✅ 資料集格式轉換完成！


In [5]:
# 檢視處理後的資料集範例
print("\033[1m處理後的資料集範例：\033[0m")
sample = processed_dataset[randint(0, len(processed_dataset) - 1)]
print(json.dumps(sample, indent=2, ensure_ascii=False))

# 計算最長範例的 token 數
longest_example = max(processed_dataset, key=lambda x: len(x['prompt'] + x['completion']))
longest_token_count = len(tokenizer.tokenize(longest_example['prompt'] + longest_example['completion']))

print(f"\n\033[1m最長範例長度: {len(longest_example['prompt'] + longest_example['completion'])} 字元, {longest_token_count} tokens\033[0m")

# 設定 max_token_count（加上緩衝）
max_token_count = longest_token_count + 100
print(f"\033[1m使用 max_token_count: {max_token_count}\033[0m")

處理後的資料集範例：
{
  "text": "{\"metadata\": \"train\", \"tools\": [{\"type\": \"function\", \"function\": {\"name\": \"create_coffee_robot_mission\", \"description\": \"建立咖啡機器人外送任務。負責驗證訂單內容，並產生機器人任務指令 (Mock Nuwa Payload)。\", \"parameters\": {\"type\": \"object\", \"properties\": {\"baseDrink\": {\"type\": \"string\", \"enum\": [\"americano\", \"latte\", \"oat_latte\", \"milk\"], \"description\": \"基礎飲品代號，只能是: americano, latte, oat_latte, milk\"}, \"floor\": {\"type\": \"integer\", \"description\": \"送達樓層，必須介於 1 到 11 之間\"}, \"addons\": {\"type\": \"array\", \"items\": {\"type\": \"string\", \"enum\": [\"extra_espresso\", \"paper_cup\"]}, \"description\": \"加購項目清單\"}, \"quantity\": {\"type\": \"integer\", \"description\": \"數量，預設為 1\"}, \"temperature\": {\"type\": \"string\", \"enum\": [\"hot\", \"iced\"], \"description\": \"溫度，只能是 hot 或 iced，預設為 hot\"}}, \"required\": [\"baseDrink\", \"floor\"]}}}], \"messages\": [{\"role\": \"user\", \"content\": \"我想要一杯加紙杯的拿鐵，送到2樓\"}, {\"role\": \"assistan

In [6]:
# 分割訓練集與驗證集
train_dataset = processed_dataset.filter(lambda x: x['split'] == 'train')
eval_dataset = processed_dataset.filter(lambda x: x['split'] == 'eval')

print(f"📊 訓練集: {len(train_dataset)} 筆")
print(f"📊 驗證集: {len(eval_dataset)} 筆")

Filter:   0%|          | 0/699 [00:00<?, ? examples/s]

Filter:   0%|          | 0/699 [00:00<?, ? examples/s]

📊 訓練集: 625 筆
📊 驗證集: 74 筆


## 6. 測試基礎模型（微調前）

先測試 FunctionGemma 基礎模型在咖啡機器人任務上的表現。

In [ ]:
import gc
import torch
from transformers import pipeline

# 清理之前可能存在的 pipeline
gc.collect()
if torch.backends.mps.is_available():
    torch.mps.empty_cache()

# 建立推論 pipeline（使用較少記憶體的設定）
pipe = pipeline(
    "text-generation", 
    model=gemma_model, 
    tokenizer=tokenizer,
    device="mps",
    torch_dtype=torch.float32
)

# 測試提示
user_prompt = "幫我送一杯熱美式到五樓"

# 從資料集取得工具定義
tools = json.loads(dataset[0]['text'])['tools']

messages = [
    {"role": "user", "content": user_prompt}
]

prompt = tokenizer.apply_chat_template(
    messages,
    tools=tools,
    tokenize=False,
    add_generation_prompt=True
)

print(f"\n\033[1mPrompt:\033[0m {user_prompt}")
output = pipe(prompt, max_new_tokens=max_token_count)
model_output = output[0]['generated_text'][len(prompt):].strip()
print(f"\n\033[1m基礎模型輸出:\033[0m {model_output}")

# 測試完釋放 pipeline
del pipe
gc.collect()
if torch.backends.mps.is_available():
    torch.mps.empty_cache()
print("\n✅ 測試完成，已釋放記憶體")

Device set to use mps:0



Prompt: 幫我送一杯熱美式到五樓

基礎模型輸出: I can help with that. Could you please specify the exact floor number for the delivery to the 5th floor?


In [ ]:
# 跳過這個測試以節省記憶體，訓練完再測試
# 從訓練集中選取一筆資料測試
# rand_idx = randint(0, len(train_dataset) - 1)
# test_sample = train_dataset[rand_idx]
# ...

print("⏭️ 跳過額外測試以保留記憶體給訓練使用")
print("💡 訓練完成後會進行完整測試")


預期輸出: <start_function_call>call:create_coffee_robot_mission{                    {"baseDrink": "milk", "floor": 1, "temperature": "hot", "quantity": 1, "addons": ["paper_cup"]}}<end_function_call><start_function_response>

實際輸出: <start_function_call>call:create_coffee_robot_mission{addons:[<escape>extra_espresso<escape>],baseDrink:[<escape>americano<escape>,<escape>latte<escape>,<escape>oat_latte<escape>,<escape>milk<escape>],floor:1,quantity:1,temperature:<escape>hot<escape>}<end_function_call><start_function_response>


## 7. 配置微調訓練

使用 TRL 的 SFTConfig 配置訓練參數。

In [ ]:
import torch
import gc
from transformers import AutoModelForCausalLM
from trl import SFTConfig

# 清理記憶體
gc.collect()
if torch.backends.mps.is_available():
    torch.mps.empty_cache()

# 輸出目錄
output_dir = "./coffee-robot-functiongemma"

# 重新載入 tokenizer
tokenizer = AutoTokenizer.from_pretrained(gemma_model)

# ======== M4 Pro 48GB 優化配置 ========
args = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=3,                               # 訓練輪數
    per_device_train_batch_size=1,                    # ⬇️ 降低批次大小以減少記憶體
    per_device_eval_batch_size=1,                     # ⬇️ 降低評估批次大小
    gradient_accumulation_steps=16,                   # ⬆️ 增加梯度累積補償小批次
    logging_strategy="steps",
    eval_strategy="steps",
    eval_steps=50,
    logging_steps=50,
    save_strategy="epoch",
    learning_rate=1e-5,                               # 學習率
    lr_scheduler_type="cosine",
    max_length=max_token_count,
    gradient_checkpointing=True,                      # ✅ 保持啟用以節省記憶體
    gradient_checkpointing_kwargs={"use_reentrant": False},  # 更節省記憶體
    packing=False,
    optim="adamw_torch",                              # 🔧 改用標準 AdamW（MPS 相容性更好）
    bf16=False,                                       # 🔧 MPS 不支援 bf16
    fp16=False,                                       # 🔧 MPS 對 fp16 支援有限，用 fp32 更穩定
    completion_only_loss=True,                        # 只計算 completion 的 loss
    report_to="none",
    dataloader_pin_memory=False,                      # 🔧 MPS 不需要 pin_memory
    dataloader_num_workers=0,                         # 🔧 避免多進程問題
    max_grad_norm=1.0,                                # 梯度裁剪防止記憶體爆炸
)

# 載入基礎模型 - 針對 Apple Silicon 優化
base_model = AutoModelForCausalLM.from_pretrained(
    gemma_model,
    device_map="mps",                                 # 🔧 明確指定 MPS
    torch_dtype=torch.float32,                        # 🔧 使用 fp32 以獲得最佳 MPS 相容性
    attn_implementation='eager',
    low_cpu_mem_usage=True,                           # 🔧 減少 CPU 記憶體使用
)

base_model.config.pad_token_id = tokenizer.pad_token_id

# 啟用記憶體高效設定
base_model.config.use_cache = False                   # 訓練時關閉 KV cache

print("✅ 訓練配置完成！")
print(f"📱 Device: {next(base_model.parameters()).device}")
print(f"📊 Effective batch size: {args.per_device_train_batch_size * args.gradient_accumulation_steps}")

`torch_dtype` is deprecated! Use `dtype` instead!


✅ 訓練配置完成！


## 8. 開始訓練

In [ ]:
import gc
import torch
from trl import SFTTrainer

# 訓練前清理記憶體
gc.collect()
if torch.backends.mps.is_available():
    torch.mps.empty_cache()

# 建立訓練器
trainer = SFTTrainer(
    model=base_model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

print("🚀 開始訓練...")
print(f"📊 訓練集大小: {len(train_dataset)}")
print(f"📊 驗證集大小: {len(eval_dataset)}")

# 開始訓練
trainer.train()

# 訓練完成後清理記憶體
gc.collect()
if torch.backends.mps.is_available():
    torch.mps.empty_cache()

# 儲存模型
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"\n✅ 微調模型已儲存至: {output_dir}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Adding EOS to train dataset:   0%|          | 0/625 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/625 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/625 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/74 [00:00<?, ? examples/s]

...

Truncating eval dataset:   0%|          | 0/74 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 2, 'pad_token_id': 0}.
/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


🚀 開始訓練...


Step,Training Loss,Validation Loss


/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/homebrew/Caskroom/miniconda/base/envs/ml-env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


## 9. 視覺化訓練結果

In [ ]:
import matplotlib.pyplot as plt

# 取得訓練記錄
log_history = trainer.state.log_history

# 提取 loss
train_losses = [log["loss"] for log in log_history if "loss" in log]
epoch_train = [log["epoch"] for log in log_history if "loss" in log]
eval_losses = [log["eval_loss"] for log in log_history if "eval_loss" in log]
epoch_eval = [log["epoch"] for log in log_history if "eval_loss" in log]

# 繪製圖表
plt.figure(figsize=(10, 6))
plt.plot(epoch_train, train_losses, label="Training Loss", marker='o', markersize=3)
plt.plot(epoch_eval, eval_losses, label="Validation Loss", marker='s', markersize=3)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Coffee Robot MCP - Training and Validation Loss")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 10. 測試微調後的模型

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# 載入微調後的模型
trained_model = AutoModelForCausalLM.from_pretrained(output_dir, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(output_dir)

# 建立 pipeline
pipe_trained = pipeline("text-generation", model=trained_model, tokenizer=tokenizer)
pipe_base = pipeline("text-generation", model=gemma_model, device_map="auto")

# 從資料集取得工具定義
tools = json.loads(dataset[0]['text'])['tools']

print("✅ 模型載入完成！")

In [ ]:
# 測試多個提示
test_prompts = [
    "幫我送一杯熱美式到五樓",
    "我要三杯冰拿鐵送到 11 樓，都要加濃縮",
    "好累喔，來杯燕麥奶拿鐵提神，我在七樓",
    "一杯鮮乳，用紙杯裝，送到三樓",
]

for user_prompt in test_prompts:
    messages = [{"role": "user", "content": user_prompt}]
    
    prompt = tokenizer.apply_chat_template(
        messages,
        tools=tools,
        tokenize=False,
        add_generation_prompt=True
    )
    
    output_trained = pipe_trained(prompt, max_new_tokens=max_token_count)
    output_base = pipe_base(prompt, max_new_tokens=max_token_count)
    
    model_output_trained = output_trained[0]['generated_text'][len(prompt):].strip()
    model_output_base = output_base[0]['generated_text'][len(prompt):].strip()
    
    print(f"\n{'='*60}")
    print(f"\033[1m使用者輸入:\033[0m {user_prompt}")
    print(f"\033[1m微調模型輸出:\033[0m {model_output_trained}")
    print(f"\033[1m基礎模型輸出:\033[0m {model_output_base}")

## 11. 評估模型

In [ ]:
import re
import pandas as pd
from tqdm import tqdm

def extract_function_call(model_output):
    """解析 Function Call 輸出"""
    results = []
    call_pattern = r"<start_function_call>(.*?)<end_function_call>"
    raw_calls = re.findall(call_pattern, model_output, re.DOTALL)
    
    for raw_call in raw_calls:
        if not raw_call.strip().startswith("call:"):
            continue
        
        try:
            pre_brace, args_segment = raw_call.split("{", 1)
            function_name = pre_brace.replace("call:", "").strip()
            
            args_content = args_segment.strip()
            if args_content.endswith("}"):
                args_content = args_content[:-1]
            
            arguments = {}
            arg_pattern = r"(?P<key>[^:,]*?):<escape>(?P<value>.*?)<escape>"
            arg_matches = re.finditer(arg_pattern, args_content, re.DOTALL)
            
            for match in arg_matches:
                key = match.group("key").strip()
                value = match.group("value")
                arguments[key] = value
            
            results.append({
                "function": {
                    "name": function_name,
                    "arguments": arguments
                }
            })
        except ValueError:
            continue
    
    return results

def evaluate_model(dataset, pipe, desc="Evaluating"):
    """評估模型在資料集上的表現"""
    logs = []
    
    for i in tqdm(range(len(dataset)), desc=desc):
        orig_data = dataset[i]['text']
        messages = json.loads(orig_data)['messages']
        user_message = messages[0]['content']
        assistant_message = messages[1]
        input_prompt = dataset[i]['prompt']
        
        output = pipe(input_prompt, max_new_tokens=max_token_count)
        model_output_only = output[0]['generated_text'][len(input_prompt):].strip()
        
        logs.append({
            "user": user_message,
            "target_fc": assistant_message.get('tool_calls', []),
            "output_fc": extract_function_call(model_output_only),
        })
    
    return logs

def calculate_score(logs):
    """計算評估分數"""
    df = pd.DataFrame.from_records(logs)
    
    df['target_names'] = df['target_fc'].apply(lambda x: [fc['function']['name'] for fc in x] if x else [])
    df['output_names'] = df['output_fc'].apply(lambda x: [fc['function']['name'] for fc in x] if x else [])
    df['correct_names'] = df['target_names'] == df['output_names']
    
    return df, df['correct_names'].mean()

print("✅ 評估函數定義完成！")

In [ ]:
# 評估基礎模型
print("📊 評估基礎模型...")
base_logs = evaluate_model(
    eval_dataset,
    pipeline("text-generation", model=gemma_model, device_map="auto", temperature=0.001),
    desc="Base Model"
)
base_df, base_score = calculate_score(base_logs)

print(f"\n\033[1m基礎模型準確率:\033[0m {base_score:.2%}")

In [ ]:
# 評估微調模型
print("📊 評估微調模型...")
trained_logs = evaluate_model(
    eval_dataset,
    pipeline("text-generation", model=trained_model, tokenizer=tokenizer, temperature=0.001),
    desc="Fine-tuned Model"
)
trained_df, trained_score = calculate_score(trained_logs)

print(f"\n\033[1m微調模型準確率:\033[0m {trained_score:.2%}")

In [ ]:
# 比較結果
print("\n" + "="*50)
print("📊 模型比較結果")
print("="*50)
print(f"基礎模型準確率:   {base_score:.2%}")
print(f"微調模型準確率:   {trained_score:.2%}")
print(f"準確率提升:       {(trained_score - base_score):.2%}")
print("="*50)

# 儲存評估結果
base_df.to_json('eval_base_model.json', orient='records', force_ascii=False)
trained_df.to_json('eval_trained_model.json', orient='records', force_ascii=False)
print("\n✅ 評估結果已儲存！")

## 12. 上傳模型至 Hugging Face Hub（可選）

In [ ]:
from huggingface_hub import ModelCard, whoami

# 設定模型名稱
model_name = "coffee-robot-mcp"

# 取得使用者名稱
username = whoami()['name']
hf_repo_id = f"{username}/functiongemma-270m-it-{model_name}"

# 上傳模型
print(f"📤 上傳模型至: {hf_repo_id}")
repo_url = trained_model.push_to_hub(hf_repo_id, create_repo=True, commit_message="Upload coffee robot MCP model")
tokenizer.push_to_hub(hf_repo_id)

# 建立 Model Card
card_content = f"""
---
base_model: {gemma_model}
tags:
- function-calling
- coffee-robot
- mcp
- gemma
language:
- zh
---

# Coffee Robot MCP - FunctionGemma 270M

基於 `{gemma_model}` 微調的咖啡機器人外送任務模型。

## 功能
- 飲品選擇: americano, latte, oat_latte, milk
- 樓層配送: 1-11 樓
- 溫度選擇: hot, iced
- 加購選項: extra_espresso, paper_cup

## 使用方式
```python
from transformers import pipeline

pipe = pipeline("text-generation", model="{hf_repo_id}")
```
"""

card = ModelCard(card_content)
card.push_to_hub(hf_repo_id)

print(f"\n✅ 模型已上傳至: {repo_url}")

## 13. 總結

🎉 **恭喜！你已經完成 FunctionGemma 的微調！**

### 完成的工作
1. ✅ 載入並處理 `ollama_mcp_dataset.jsonl` 資料集
2. ✅ 微調 FunctionGemma 270M 模型
3. ✅ 評估模型表現
4. ✅ 儲存微調後的模型

### 下一步
- 使用更多資料繼續訓練
- 調整超參數優化效果
- 部署模型至生產環境